In [1]:
from time import mktime
import datetime
from datetime import datetime
import os.path
import pandas as pd
import os
import subprocess
import time
import statistics
global path
global path_list
global i
global dfpaths

fname = '2025.05.07_19.22_(12062)_GDrive_All_Folders_1' 
pathfolder = '2025.05.07/'
fnamepath = '/Users/rommellmontenegro/Desktop/Civil Suit*/5 - Mobile Graphic & Video Media/RICO Album Tables/ADMIN-Working Files/GDrive Folder Lists/GDrive All Folders/' + pathfolder + fname +'.xlsx'
df = pd.read_excel(fnamepath)

currentDateTime = datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y") #except: 2022
date.strftime("%Y")
starttime = datetime.now()
startR = str(starttime.strftime('%H.%M'))
d = datetime.now()
only_date, only_time = d.date(), d.time()
only_date = only_date.strftime('%Y.%m.%d')
only_time = only_time.strftime('%H.%M')
procTS = str(only_date) + "_" + str(only_time)
Count = 1
ProcRates = []
Procper = 0
dfpaths = []

def PathFinder(fileId):    
    #Source: https://stackoverflow.com/questions/57618533/google-drive-api-how-can-i-find-the-path-of-a-file
    import os.path
    import pandas as pd
    from google.auth.transport.requests import Request
    from google.oauth2.credentials import Credentials
    from google_auth_oauthlib.flow import InstalledAppFlow
    from googleapiclient.discovery import build
    from googleapiclient.errors import HttpError
    from httplib2 import Http
    from httplib2 import Http
    from apiclient.http import MediaFileUpload,MediaIoBaseDownload
    import io
    global path
    global path_list
    global dfpaths
    global i
    global processtimeR
    global AVGProcRateR
    from time import mktime
    import datetime
    from datetime import datetime
    d = datetime.now()
    Total = len(df)
    TotalR = str(len(df))
    
    # If modifying these scopes, delete the file token.json.
    SCOPES = ['https://www.googleapis.com/auth/drive']

    creds = None
      # The file token.json stores the user's access and refresh tokens, and is
      # created automatically when the authorization flow completes for the first
      # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
      # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
          creds.refresh(Request())
        else:
          flow = InstalledAppFlow.from_client_secrets_file(
              "client_secret_833249459653-01pcjla0c9re30fgr2dfipcgg5lgq6j0.apps.googleusercontent.com.json", SCOPES
          )
          creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
          token.write(creds.to_json())

    # Call the Drive v3 API
    drive = build("drive", "v3", credentials=creds)

    tree = []  # Result
    file = drive.files().get(fileId=fileId, fields='id, name, parents').execute()

    parent = file.get('parents')
    if parent:
        while True:
            folder = drive.files().get(
                fileId=parent[0], fields='id, name, parents').execute()
            parent = folder.get('parents')
            if parent is None:
                break
            tree.append([folder.get('name')])
            rev = tree[::-1]

            path_list = [
            x
            for xs in rev
            for x in xs
            ]

    separator = "/"
    path = separator.join(path_list)

    endtime = datetime.now()
    processtime = (datetime.now() - starttime).total_seconds()
    if processtime > 3600:
        processtimeR = str(round(processtime/3600,0)) + ' Hour(s) and ' + str(round((processtime%3600)*60,2)) + ' Minutes'
    elif processtime > 60:
        processtimeR = str(round(processtime/60,2)) + ' Minutes'
    else:
        processtimeR = str(round(processtime,2)) + ' Seconds'

    ProcRate = Count/(processtime)
    ProcRateR = round(ProcRate,2)
    ProcRates.append(ProcRate)

    try: AVGProcRateR = round(sum(ProcRates) / len(ProcRates),2) 
    except ZeroDivisionError: AVGProcRateR = round(sum(ProcRates) / 1 ,2)

    Procremaining = Total - Count
    Procper = Count/Total
    ProcperR = round(Procper*100,2)
    try: Timeremaining = round(Procremaining/statistics.mean(ProcRates),2)
    except ZeroDivisionError: Timeremaining = round((Procremaining/1),2)
    
    if Timeremaining > 7200:
        TimeremainingR = str(round(Timeremaining/3600,0)) + ' Hour(s) and ' + str(abs(round((Timeremaining-(round(Timeremaining/3600,0)*3600))/60,2))) + ' Minutes'
    elif Timeremaining > 3600:
        TimeremainingR = str(round(Timeremaining/3600,0)) + ' Hour(s) and ' + str(round((Timeremaining%3600)*60,2)) + ' Minutes'
    elif Timeremaining > 60:
        TimeremainingR = str(round(Timeremaining/60,2)) + ' Minutes'
    else:
        TimeremainingR = str(round(Timeremaining,2)) + ' Seconds'

    print("Processed " + str(Count) + " files in " + processtimeR + ". " + str(ProcperR) + "% completed of " + TotalR + " and "+ TimeremainingR + ' left at ' + 
          str(d.strftime("%H:%M:%S")) + ".                                                       ", end='\r')
    return path

for i in df['id']:
    path = PathFinder(i)
    dfpaths.append(path)
    Count += 1
    StrCount = str(Count)
    
df['path'] = dfpaths

d = datetime.now()
only_date, only_time = d.date(), d.time()
only_date = only_date.strftime('%Y.%m.%d')
only_time = only_time.strftime('%H.%M')
column_order = ['name', 'id', 'webViewLink','path']
tabname = '1 to 1000'
Local = "/Users/rommellmontenegro/Desktop/Civil Suit*/5 - Mobile Graphic & Video Media/RICO Album Tables/Combine/Google Path/"
HD_Dest = Local + only_date + "_" + only_time + "_" + tabname + "_GDrivePath.xlsx"

with pd.ExcelWriter(HD_Dest) as writer:
    df[column_order].to_excel(writer, sheet_name=tabname)
print("Processed " + str(Count) + " files in " + processtimeR + ". Average time per file: " + str(AVGProcRateR) + "Started at: " + startR + " and Ended at: "+ only_time +  ".") 

Processed 1002 files in 36.98 Minutes. Average time per file: 0.45Started at: 19.28 and Ended at: 20.05.                                                 
